In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install python-dotenv
# import kagglehub
# import os
# from dotenv import load_dotenv

# # Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
# dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# # Sprawdź, czy plik istnieje przed próbą jego wczytania
# if os.path.exists(dotenv_path):
#     load_dotenv(dotenv_path)

#     # Pobierz zmienne środowiskowe
#     kaggle_login = os.getenv('KAGGLE_LOGIN')
#     kaggle_key = os.getenv('KAGGLE_KEY')

#     # # Możesz teraz użyć kaggle_login i kaggle_key
#     # print(f"KAGGLE_LOGIN: {kaggle_login}")
#     # print(f"KAGGLE_KEY: {kaggle_key}")
# else:
#     print(f"Plik .env nie został znaleziony w {dotenv_path}")
# os.environ['KAGGLE_USERNAME'] = kaggle_login
# os.environ['KAGGLE_KEY'] = kaggle_key

# kagglehub.login()


In [2]:
# print('Importing data sources...')

# competition_path = kagglehub.competition_download('digit-recognizer')

# print('Data source import complete.')

In [3]:
# # List files in the downloaded directory
# print(f"Files in the downloaded directory ({competition_path}):")
# for dirname, _, filenames in os.walk(competition_path):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# Wczytanie danych
# ds_train = pd.read_csv(os.path.join(competition_path, "train.csv"))
# ds_test = pd.read_csv(os.path.join(competition_path, "test.csv"))
ds_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
ds_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
print(f'Train shape: {ds_train.shape}')
print(f'Test shape: {ds_test.shape}')

2025-07-10 19:40:57.680624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752176457.880967      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752176457.933568      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train shape: (42000, 785)
Test shape: (28000, 784)


In [5]:
from sklearn.model_selection import train_test_split

X_train_full = ds_train.drop('label', axis=1)
y_train_full = ds_train['label']
y_train = y_train_full

# Normalizacja pikseli

X_train = X_train_full / 255
X_test = ds_test / 255

# # Podział na zbiór treningowy i walidacyjny
# X_train, X_valid = X_train_full[:-2000], X_train_full[-2000:]
# y_train, y_valid = y_train_full[:-2000], y_train_full[-2000:]

In [6]:
# Ustaw ziarno losowości
import random

SEED = 42
# os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [7]:
# Dodanie wymiaru kanału dla "Splotowej Sieci Neuronowej"
X_train = X_train.values.reshape(-1, 28, 28, 1)
# X_valid = X_valid.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)


In [8]:
y_train = y_train.astype('int')

In [9]:
X=X_train
y=y_train

In [10]:
# Model
import tensorflow as tf
from tensorflow.keras import layers, models, activations
from tensorflow.keras import backend as K

# Custom Swish activation (alternatywnie: use tf.nn.swish)
def swish(x):
    return x * K.sigmoid(x)

# Squeeze-and-Excitation Block
def se_block(input_tensor, reduction=16):
    channels = input_tensor.shape[-1]
    se = layers.GlobalAveragePooling2D()(input_tensor)
    se = layers.Dense(channels // reduction, activation='relu')(se)
    se = layers.Dense(channels, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, channels))(se)
    return layers.Multiply()([input_tensor, se])

# Model
def build_advanced_model():
    inputs = layers.Input(shape=(28, 28, 1))
    
    x = layers.Conv2D(32, (3,3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = layers.Conv2D(32, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)
    x = se_block(x)  # Dodanie SE blocka
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Conv2D(64, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    x = layers.Conv2D(64, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)
    x = se_block(x)
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# === 3. Konfiguracja augmentacji ===
# Tylko dla danych treningowych — nie dla walidacyjnych!
datagen = ImageDataGenerator(
    rotation_range=10,        # losowy obrót w zakresie [-10°, 10°]
    zoom_range=0.1,           # przybliżenie losowo w zakresie [0.9, 1.1]
    width_shift_range=0.1,    # przesunięcie poziome (maks. 10%)
    height_shift_range=0.1    # przesunięcie pionowe (maks. 10%)
)

**Stratified Cross Validation**


In [12]:
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []
models_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold + 1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = build_advanced_model()
    
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,             # zatrzymaj po 5 epokach bez poprawy
        restore_best_weights=True
    )

    datagen.fit(X_train)

    #Trening
    model.fit(
        datagen.flow(X_train, y_train, batch_size=64),  # dane augmentowane w locie
        validation_data=(X_val, y_val),                 # bez augmentacji
        epochs=30,
        callbacks=[early_stop],
        verbose=1
    )

    models_list.append(model)  # zapisujemy model z tego folda
    # Ocena
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"✅ Fold {fold+1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

# Średnia dokładność
mean_acc = np.mean(fold_accuracies)
print(f"\n📊 Średnia dokładność z {skf.n_splits} foldów: {mean_acc:.4f}")


📂 Fold 1


I0000 00:00:1752176532.179166      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1752176540.166216      99 service.cc:148] XLA service 0x7f030c01bfb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752176540.167213      99 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752176540.847232      99 cuda_dnn.cc:529] Loaded cuDNN version 90300


 11/525 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.2537 - loss: 2.2000  

...

525/525 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.7973 - loss: 0.6496 - val_accuracy: 0.9711 - val_loss: 0.1021
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9642 - loss: 0.1204 - val_accuracy: 0.9857 - val_loss: 0.0473
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9728 - loss: 0.0919 - val_accuracy: 0.9833 - val_loss: 0.0565
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9752 - loss: 0.0794 - val_accuracy: 0.9908 - val_loss: 0.0312
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9807 - loss: 0.0633 - val_accuracy: 0.9895 - val_loss: 0.0376
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9821 - loss: 0.0573 - val_accuracy: 0.9911 - val_loss: 0.0296
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9825 - loss: 0.0568 - val_accuracy: 0.9926 - val_loss: 0.0238
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9858 - loss: 0.0450 - val_accurac

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


525/525 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.7896 - loss: 0.6696 - val_accuracy: 0.9798 - val_loss: 0.0822
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9655 - loss: 0.1206 - val_accuracy: 0.9789 - val_loss: 0.0634
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9734 - loss: 0.0948 - val_accuracy: 0.9910 - val_loss: 0.0295
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9743 - loss: 0.0769 - val_accuracy: 0.9937 - val_loss: 0.0248
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9815 - loss: 0.0598 - val_accuracy: 0.9889 - val_loss: 0.0362
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9820 - loss: 0.0609 - val_accuracy: 0.9923 - val_loss: 0.0262
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9845 - loss: 0.0518 - val_accuracy: 0.9926 - val_loss: 0.0292
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9869 - loss: 0.0419 - val_accurac

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


525/525 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.7903 - loss: 0.6610 - val_accuracy: 0.9662 - val_loss: 0.1170
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9629 - loss: 0.1281 - val_accuracy: 0.9902 - val_loss: 0.0314
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9724 - loss: 0.0942 - val_accuracy: 0.9881 - val_loss: 0.0375
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9755 - loss: 0.0799 - val_accuracy: 0.9890 - val_loss: 0.0337
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9802 - loss: 0.0630 - val_accuracy: 0.9910 - val_loss: 0.0260
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9793 - loss: 0.0674 - val_accuracy: 0.9908 - val_loss: 0.0266
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.9832 - loss: 0.0531 - val_accuracy: 0.9911 - val_loss: 0.0277
Epoch 8/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9835 - loss: 0.0546 - val_accurac

In [15]:
def predict_with_tta(model, image, datagen, n_augments=5):
    preds = []
    image = np.expand_dims(image, axis=0)  # shape: (1, 28, 28, 1)

    for _ in range(n_augments):
        # Losowa augmentacja (ImageDataGenerator działa na batchach)
        aug_iter = datagen.flow(image, batch_size=1, shuffle=False)
        augmented_img = next(aug_iter)
        pred = model.predict(augmented_img, verbose=0)
        preds.append(pred[0])  # pred shape: (1, 10)

    return np.mean(preds, axis=0)  # shape: (10,)


In [20]:
# === 6. Ensemble predykcji ===
tta_datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

n_augments = 5
all_preds = np.zeros((X_test.shape[0], 10))

for model_idx, model in enumerate(models_list):
    print(f"🔍 TTA predykcje dla modelu {model_idx+1}...")

    tta_preds = np.zeros((n_augments, X_test.shape[0], 10))  # shape: (n_augments, N, 10)

    for i in range(n_augments):
        print(f"  ↪️ Augmentacja {i+1}/{n_augments}")

        # Augmentujemy każdy obraz ręcznie
        augmented_batch = np.array([
            tta_datagen.random_transform(img) for img in X_test
        ])

        preds = model.predict(augmented_batch, batch_size=256, verbose=0)
        tta_preds[i] = preds  # OK — shape: (28000, 10)

    mean_preds = np.mean(tta_preds, axis=0)  # shape: (28000, 10)
    all_preds += mean_preds

ensemble_preds = np.argmax(all_preds / len(models_list), axis=1)


🔍 TTA predykcje dla modelu 1...
  ↪️ Augmentacja 1/5
  ↪️ Augmentacja 2/5
  ↪️ Augmentacja 3/5
  ↪️ Augmentacja 4/5
  ↪️ Augmentacja 5/5
🔍 TTA predykcje dla modelu 2...
  ↪️ Augmentacja 1/5
  ↪️ Augmentacja 2/5
  ↪️ Augmentacja 3/5
  ↪️ Augmentacja 4/5
  ↪️ Augmentacja 5/5
🔍 TTA predykcje dla modelu 3...
  ↪️ Augmentacja 1/5
  ↪️ Augmentacja 2/5
  ↪️ Augmentacja 3/5
  ↪️ Augmentacja 4/5
  ↪️ Augmentacja 5/5
🔍 TTA predykcje dla modelu 4...
  ↪️ Augmentacja 1/5
  ↪️ Augmentacja 2/5
  ↪️ Augmentacja 3/5
  ↪️ Augmentacja 4/5
  ↪️ Augmentacja 5/5
🔍 TTA predykcje dla modelu 5...
  ↪️ Augmentacja 1/5
  ↪️ Augmentacja 2/5
  ↪️ Augmentacja 3/5
  ↪️ Augmentacja 4/5
  ↪️ Augmentacja 5/5


In [ ]:
# history = model.fit(X_train, y_train,
#                     batch_size=64,
#                     epochs=30,
#                     validation_data=(X_valid, y_valid),
#                     shuffle=False,
#                     callbacks=[early_stop])


# def plot_history(history):
#     plt.figure(figsize=(12, 4))

#     # Accuracy
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['accuracy'], label='train acc')
#     if 'val_accuracy' in history.history:
#         plt.plot(history.history['val_accuracy'], label='val acc')
#     plt.title('Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.legend()

#     # Loss
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['loss'], label='train loss')
#     if 'val_loss' in history.history:
#         plt.plot(history.history['val_loss'], label='val loss')
#     else:
#         print("⚠️ Nie znaleziono 'val_loss' — czy podałeś validation_data?")
#     plt.title('Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# plot_history(history)

In [ ]:
# y_pred_proba = model.predict(X_test)
# y_pred = np.argmax(y_pred_proba, axis=1)

In [23]:
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(ensemble_preds) + 1),
    "Label": ensemble_preds
})

submission.to_csv("09_submission.csv", index=False)